# Juntando as Peças 🧩

## (Fontes de Dados 🚰 ➕ Tratamento dos Dados 🪢) <sup>Alto Desempenho 🚀 🧞‍♂️</sup>

### Funções Auxiliares

In [1]:
import os, pandas as pd, re
from datetime import date

def build_path(subfolder = 'clean'):
    folderpath = os.path.join(os.getcwd(), os.pardir, 
                              'project', 'data', subfolder)
    folderpath = os.path.abspath(folderpath)
    if not os.path.exists(folderpath): 
        os.makedirs(folderpath)
    return folderpath

def load_data():
    data = {}   
    folderpath = build_path()
    for filename in os.listdir(folderpath):
        if filename.endswith('.json'):
            filepath = os.path.join(build_path(), filename)
            data[filename.split('.')[0]] = pd.read_json(filepath)
    return data

def merge_state_vaccine_shot(data):
    vacinacao_estados = pd.merge(data['estados_codigos'],
                                 data['estados_georreferenciamento'],
                                 left_on='id',
                                 right_on='codigo_uf',
                                 how='inner')

    vacinacao_estados = pd.merge(vacinacao_estados,
                                 data['estados_vacinacao'],
                                 left_on='sigla',
                                 right_on='estado_key',
                                 how='inner')

    vacinacao_estados = vacinacao_estados.drop(columns=['estado_key', 'sigla', 'codigo_uf'])

    vacinacao_estados['vacinacao_key'] = \
        pd.to_datetime(vacinacao_estados['vacinacao_key'], unit='ms').dt.date

    vacinacao_estados = vacinacao_estados[date(2020, 1, 1) < vacinacao_estados['vacinacao_key']]
    vacinacao_estados = vacinacao_estados[vacinacao_estados['vacinacao_key'] < date.today()]
    vacinacao_estados = vacinacao_estados.sort_values(by=['vacinacao_key'])
    return vacinacao_estados

def merge_city_vaccine_shot(data):
    vacinacao_municipios = pd.merge(data['municipios_codigos'],
                                 data['municipios_georreferenciamento'],
                                 left_on='id',
                                 right_on='codigo_ibge',
                                 how='inner')

    vacinacao_municipios['codigo_ibge'] = \
        vacinacao_municipios['codigo_ibge'].astype(str).str[:-1].astype(int)

    vacinacao_municipios = pd.merge(vacinacao_municipios,
                                    data['municipios_vacinacao'],
                                    left_on='codigo_ibge',
                                    right_on='municipio_key',
                                    how='left')

    vacinacao_municipios = vacinacao_municipios.drop(columns=['id', 'municipio_key'])

    vacinacao_municipios['vacinacao_key'] = \
        pd.to_datetime(vacinacao_municipios['vacinacao_key'], unit='ms', errors='ignore').dt.date

    vacinacao_municipios = vacinacao_municipios[date(2020, 1, 1) < vacinacao_municipios['vacinacao_key']]
    vacinacao_municipios = vacinacao_municipios[vacinacao_municipios['vacinacao_key'] < date.today()]
    vacinacao_municipios = vacinacao_municipios.sort_values(by=['vacinacao_key'])
    return vacinacao_municipios

def merge_city_infecction(data):
    infeccoes_municipios = pd.merge(data['municipios_codigos'],
                                    data['municipios_georreferenciamento'],
                                    left_on='id',
                                    right_on='codigo_ibge',
                                    how='inner')

    infeccoes_municipios['codigo_ibge'] = \
        infeccoes_municipios['codigo_ibge'].astype(str).str[:-1].astype(int)

    infeccoes_municipios = pd.merge(infeccoes_municipios,
                                    data['ALL_HIST_PAINEL_COVID'],
                                    left_on='codigo_ibge',
                                    right_on='codmun',
                                    how='inner')

    infeccoes_municipios = infeccoes_municipios.drop(columns=['id', 'codmun'])
    infeccoes_municipios = infeccoes_municipios.sort_values(by=['codigo_ibge', 'data'])
    infeccoes_municipios = infeccoes_municipios[infeccoes_municipios['obitosNovos'] >= 0]
    return infeccoes_municipios

def merge_state_infections_characteristics(data):
    covid_infections_by_city = \
        data['ALL_HIST_PAINEL_COVID'].groupby('codmun').agg({'casosNovos': 'sum', 'obitosNovos': 'sum'})
    covid_infections_by_city['codestado'] = covid_infections_by_city.index.astype('str').str.slice(0,2)
    covid_infections_by_city['regiao'] = covid_infections_by_city.index.astype('str').str.slice(0,1)
    covid_infections_by_city['regiao'] = covid_infections_by_city['regiao'].map(
        {'1': 'Norte', '2': 'Nordeste', '3': 'Sudeste', '4': 'Sul', '5': 'Centro-Oeste'}
    )
    covid_infections_by_city['codestado'] = covid_infections_by_city['codestado'].astype('int')
    covid_infections_by_state = \
        covid_infections_by_city.groupby('codestado').\
            agg({'casosNovos': 'sum', 'obitosNovos': 'sum', 'regiao': 'first'})
    
    estados_caracteristicas = data['estados_caracteristicas'].pivot(index='state_state', 
                                                                    columns='characteristics_label', 
                                                                    values='characteristics_value')
    
    estados_caracteristicas.index = estados_caracteristicas.index.str.upper()
    estados_caracteristicas['População residente'] = \
        estados_caracteristicas['População residente'].\
            map(lambda x: re.sub('\D', '', x)).astype('int')
    estados_caracteristicas['Densidade demográfica'] = \
        estados_caracteristicas['Densidade demográfica'].\
            map(lambda x: x.replace(',', '.').rstrip(' hab/km²')).astype('float')
    estados_caracteristicas['IDH'] = \
        estados_caracteristicas['IDHÍndice de desenvolvimento humano'].\
            map(lambda x: x.replace(',', '.')).astype('float')
    estados_caracteristicas['Total de veículos'] = \
        estados_caracteristicas['Total de veículos'].\
            map(lambda x: x.replace('.', '').rstrip(' veículos')).astype('int')
    estados_caracteristicas['Veículos per capita'] = \
        estados_caracteristicas['Total de veículos']/estados_caracteristicas['População residente']
    estados_caracteristicas['Rendimento mensal domiciliar per capita'] = \
        estados_caracteristicas['Rendimento mensal domiciliar per capita'].\
            map(lambda x: x.replace('.', '').rstrip(' R$')).astype('int')
  
    estados_caracteristicas = pd.merge(estados_caracteristicas,
                                       data['estados_codigos'],
                                       left_on='state_state',
                                       right_on='sigla',
                                       how='inner')
    
    estados_caracteristicas = pd.merge(estados_caracteristicas,
                                       covid_infections_by_state,
                                       left_on='id',
                                       right_on='codestado',
                                       how='inner')
    
    estados_caracteristicas['TaxaLetalidade'] = \
        estados_caracteristicas['obitosNovos']/estados_caracteristicas['casosNovos']
    
    estados_caracteristicas = \
        estados_caracteristicas[['id', 'nome', 'regiao', 'Densidade demográfica', 
                                 'IDH', 'Rendimento mensal domiciliar per capita', \
                                 'Veículos per capita', 'TaxaLetalidade']]

    return estados_caracteristicas

def save_merged_data(df, filename='city_infecction.json'):
    filepath = os.path.join(build_path('merge'), filename)
    df.to_json(filepath)

### Código

In [2]:
data = load_data()

state_vaccine_shot_df = merge_state_vaccine_shot(data)
city_vaccine_shot_df = merge_city_vaccine_shot(data)
city_infecction_df = merge_city_infecction(data)
state_infections_characteristics_df = merge_state_infections_characteristics(data)

save_merged_data(state_vaccine_shot_df, filename='state_vaccine_shot.json')
save_merged_data(city_vaccine_shot_df, filename='city_vaccine_shot.json')
save_merged_data(city_infecction_df, filename='city_infecction.json')
save_merged_data(state_infections_characteristics_df, 
                 filename='state_infections_characteristics.json')